In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils as U
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

### Define the Neural Network

We want to be able to set the weight of the network manually

In [2]:
class NeuralNet(nn.Module):
    def __init__(self, in_dim=2, out_dim=2):
        super(NeuralNet, self).__init__()

        self.layer1=nn.Linear(in_features=in_dim, out_features=4)
        self.layer2=nn.Linear(in_features=4, out_features=out_dim)

        self.weights_initialization()
    
    def weights_initialization(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)  # This is the default in PyTorch
                nn.init.constant_(module.bias, 0)

    def forward(self, x):
        out = self.layer1(x)
        out = F.relu(out)
        out = self.layer2(out)
        return out
    
    def get_flat_params(self):
        return U.parameters_to_vector(self.parameters())
    
    def set_flat_params(self, flat_params):
        U.vector_to_parameters(flat_params, self.parameters())

### Load Data

In [3]:
X, y = make_moons(n_samples=1000, noise=0.1, random_state=42)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

### Evolutionary Algorithm

In our case the flat parameters will be the chromosomes

In [4]:
# Initialize model and get flat parameters
model = NeuralNet()
params_vector = model.get_flat_params()
print(params_vector)


# Configuration
POPULATION_SIZE = 100
CHROMOSOME_LENGTH = len(params_vector)


tensor([-0.6845,  0.2437, -0.8973,  0.2288,  0.2638, -0.1709,  0.9182,  0.3240,
         0.0000,  0.0000,  0.0000,  0.0000, -0.0631,  0.0214,  0.5412, -0.5746,
        -0.1969, -0.7811,  0.0958,  0.0240,  0.0000,  0.0000],
       grad_fn=<CatBackward0>)


In [5]:
def generate_population(size, chromosome_length):
    return [torch.rand(chromosome_length) for _ in range(size)]

def fitness(chromosome, model):
    model.set_flat_params(chromosome)   # Update model's parameters with the chromosome
    with torch.no_grad():
        outputs = model(X_test)
        pred = outputs.argmax(dim=1)
        accuracy = (pred == y_test).float().mean().item()

    return accuracy


In [18]:
population = generate_population(POPULATION_SIZE, CHROMOSOME_LENGTH)
fitness_scores = [fitness(individual, model) for individual in population]

print(population[0])
print(fitness_scores[0])

tensor([0.5999, 0.0663, 0.3498, 0.5129, 0.2641, 0.4382, 0.0389, 0.9832, 0.7759,
        0.1934, 0.7615, 0.9589, 0.5513, 0.4209, 0.2688, 0.7039, 0.0884, 0.1870,
        0.1561, 0.6013, 0.1102, 0.2477])
0.5
